In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features_names = [
    'cement', 'water', 'superplasticizer', 'age'   
]
target_name = 'concrete_compressive_strength'
# Load the data
data = pd.read_excel('concrete_data.xlsx')

print(data.head())
# Print the headers (column names)
print("Headers:", data.columns.tolist())

print("Data shape:", data.shape)


ModuleNotFoundError: No module named 'numpy'

In [ ]:
x = data[features_names]
y = data[target_name]

scaller = StandardScaler()

def normalize(data,columns, is_test=False):
    if is_test:
        data = scaller.transform(data)
    else:
        data = scaller.fit_transform(data)
    return pd.DataFrame(data, columns=columns)
      
   
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, shuffle=True)
x_train = normalize(x_train, features_names)
x_test = normalize(x_test, is_test=True, columns=features_names)
y_train = normalize(y_train.values.reshape(-1, 1), columns=[target_name])
y_test = normalize(y_test.values.reshape(-1, 1), is_test=True, columns=[target_name])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(525, 4) (175, 4) (525, 1) (175, 1)


count    700.000000
mean      37.642514
std       17.607925
min        2.330000
25%       24.280000
50%       36.615000
75%       50.120000
max       82.600000
Name: concrete_compressive_strength, dtype: float64

In [29]:

class NeuralNetwork:
    def __init__(self,input_size, hidden_size, output_size,epochs=1000, learning_rate=0.5):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.init_weights()
        self.initialize_bais()
        
        
    def init_weights(self):
        self.weights1 = np.random.randn(self.input_size, self.hidden_size)
        self.weights2 = np.random.randn(self.hidden_size, self.output_size)
        self.bias1 = np.random.randn(1, self.hidden_size)
        
    def initialize_bais(self):
        self.hidden_layer_bais = np.random.randn((1, self.hidden_size))
        self.output_layer_bais = np.random.randn((1, self.output_size))
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward(self, x):
        hidden_layer_input = np.dot(x, self.weights1) + self.hidden_layer_bais
        hidden_layer_output = self.sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_output, self.weights2) + self.output_layer_bais
        output_layer_output = self.sigmoid(output_layer_input)
        return output_layer_output
    
    def backward(self, x, y, output):
        # Step 1 work out the error
        output_error = y - output
        output_layer_delta = self.sigmoid_derivative(output) * output_error

        # Step 3: Calculate errors for hidden neurons
        hidden_layer_input = np.dot(x, self.weights1) + self.hidden_layer_bais
        hidden_layer_output = self.sigmoid(hidden_layer_input)
        hidden_layer_error = np.dot(output_layer_delta, self.weights2.T)
        hidden_layer_delta = self.sigmoid_derivative(hidden_layer_output) * hidden_layer_error

        # Step 2: Update weights for output
        self.weights2 += self.learning_rate * np.dot(hidden_layer_output.T, output_layer_delta)
        #self.output_layer_bais += self.learning_rate * np.sum(output_layer_delta, axis=0, keepdims=True)

        # Step 4: Update weights for hidden
        self.weights1 += self.learning_rate * np.dot(x.T, hidden_layer_delta)
        #self.hidden_layer_bais += self.learning_rate * np.sum(hidden_layer_delta, axis=0, keepdims=True)
        

    def train(self, x, y):
        self.x = x
        self.y = y
        for i in range(self.epochs):
            output = self.forward(x)
            self.backward(x, y, output)
            
            error = np.mean(0.5 * ((y - output) ** 2))
            print(f"Epoch {i+1}, Error: {error}")
            if error <= 0.001: 
                print("Training stopped as error is below threshold")
                break
    
    def predict(self, x):
        return self.forward(x)
        
        
nn = NeuralNetwork(input_size=4, hidden_size=8, output_size=1, epochs=1000, learning_rate=0.5)
nn.train(x_train, y_train)

predictions = nn.predict(x_test)
mse_test = np.mean(0.5 * ((y_test - predictions) ** 2))
print(f"Test MSE: {mse_test}")        
    
        
        